In [ ]:
!pip install beautifulsoup4

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

# Extract data from Wikipedia

In [2]:
# Read the wikipedia page
with urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") as response:
    html = response.read()
    soup = BeautifulSoup(html)

In [3]:
table = soup.body.table
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

In [69]:
columns = ['Postal Code', 'Borough', 'Neighborhood']
columns

['Postal Code', 'Borough', 'Neighborhood']

In [70]:
rows = []
for tr in table.find_all('tr'):
    row = []
    for td in tr.find_all('td'):
        row.append(td.get_text()[:-1])
    
    if len(row) > 0:
        rows.append(row)
    
rows

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'],
 ['M1B', 'Scarborough', 'Malvern, Rouge'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'],
 ['M

In [83]:
df = pd.DataFrame(rows, columns = columns)
df.head()

Postal Code           Borough               Neighborhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

# Data cleaning

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
Postal Code     180 non-null object
Borough         180 non-null object
Neighborhood    180 non-null object
dtypes: object(3)
memory usage: 4.3+ KB


In [85]:
df = df[df.Borough != 'Not assigned']
df = df.reset_index(drop=True)
df.head()

Postal Code           Borough                                 Neighborhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [86]:
# update Neighbourhood to Borough if Neighbourhood = Not assigned
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood'] = df.loc[df.Neighborhood == 'Not assigned', 'Borough']
df = df.reset_index(drop=True)
df.head()

Postal Code           Borough                                 Neighborhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [87]:
df.shape

(103, 3)

# Get geograhical coordinates of neighbourhoods

In [88]:
df_crd = pd.read_csv('https://cocl.us/Geospatial_data')
df_crd.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [89]:
df_crd.shape

(103, 3)

In [92]:
df_merged = df.merge(df_crd, left_on='Postal Code', right_on='Postal Code') 
df_merged.head(12)

Postal Code           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M7A  Downtown Toronto   
5          M9A         Etobicoke   
6          M1B       Scarborough   
7          M3B        North York   
8          M4B         East York   
9          M5B  Downtown Toronto   
10         M6B        North York   
11         M9B         Etobicoke   

                                         Neighborhood   Latitude  Longitude  
0                                           Parkwoods  43.753259 -79.329656  
1                                    Victoria Village  43.725882 -79.315572  
2                           Regent Park, Harbourfront  43.654260 -79.360636  
3                    Lawrence Manor, Lawrence Heights  43.718518 -79.464763  
4         Queen's Park, Ontario Provincial Government  43.662301 -79.389494  
5             Islington Avenue, Humber Valley Village  43.667856 -79.532242  
6                                      Malvern, Rouge  43.806686 -79.194353  
7                                           Don Mills  43.745906 -79.352188  
8                     Parkview Hill, Woodbine Gardens  43.706397 -79.309937  
9                            Garden District, Ryerson  43.657162 -79.378937  
10                                          Glencairn  43.709577 -79.445073  
11  West Deane Park, Princess Gardens, Martin Grov...  43.650943 -79.554724

# Explore and cluster the neighborhoods in Toronto

In [14]:
#!pip install geopy

In [15]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [97]:
toronto_data = df_merged[df_merged.Borough.str.contains('Toronto')]
toronto_data = df_toronto.reset_index(drop=True)
toronto_data.columns = df_merged.columns
toronto_data

Postal Code           Borough  \
0          M5A  Downtown Toronto   
1          M7A  Downtown Toronto   
2          M5B  Downtown Toronto   
3          M5C  Downtown Toronto   
4          M4E      East Toronto   
5          M5E  Downtown Toronto   
6          M5G  Downtown Toronto   
7          M6G  Downtown Toronto   
8          M5H  Downtown Toronto   
9          M6H      West Toronto   
10         M5J  Downtown Toronto   
11         M6J      West Toronto   
12         M4K      East Toronto   
13         M5K  Downtown Toronto   
14         M6K      West Toronto   
15         M4L      East Toronto   
16         M5L  Downtown Toronto   
17         M4M      East Toronto   
18         M4N   Central Toronto   
19         M5N   Central Toronto   
20         M4P   Central Toronto   
21         M5P   Central Toronto   
22         M6P      West Toronto   
23         M4R   Central Toronto   
24         M5R   Central Toronto   
25         M6R      West Toronto   
26         M4S   Central Toronto   
27         M5S  Downtown Toronto   
28         M6S      West Toronto   
29         M4T   Central Toronto   
30         M5T  Downtown Toronto   
31         M4V   Central Toronto   
32         M5V  Downtown Toronto   
33         M4W  Downtown Toronto   
34         M5W  Downtown Toronto   
35         M4X  Downtown Toronto   
36         M5X  Downtown Toronto   
37         M4Y  Downtown Toronto   
38         M7Y      East Toronto   

                                         Neighborhood   Latitude  Longitude  
0                           Regent Park, Harbourfront  43.654260 -79.360636  
1         Queen's Park, Ontario Provincial Government  43.662301 -79.389494  
2                            Garden District, Ryerson  43.657162 -79.378937  
3                                      St. James Town  43.651494 -79.375418  
4                                         The Beaches  43.676357 -79.293031  
5                                         Berczy Park  43.644771 -79.373306  
6                                  Central Bay Street  43.657952 -79.387383  
7                                            Christie  43.669542 -79.422564  
8                            Richmond, Adelaide, King  43.650571 -79.384568  
9                        Dufferin, Dovercourt Village  43.669005 -79.442259  
10  Harbourfront East, Union Station, Toronto Islands  43.640816 -79.381752  
11                           Little Portugal, Trinity  43.647927 -79.419750  
12                       The Danforth West, Riverdale  43.679557 -79.352188  
13           Toronto Dominion Centre, Design Exchange  43.647177 -79.381576  
14       Brockton, Parkdale Village, Exhibition Place  43.636847 -79.428191  
15                     India Bazaar, The Beaches West  43.668999 -79.315572  
16                     Commerce Court, Victoria Hotel  43.648198 -79.379817  
17                                    Studio District  43.659526 -79.340923  
18                                      Lawrence Park  43.728020 -79.388790  
19                                           Roselawn  43.711695 -79.416936  
20                                   Davisville North  43.712751 -79.390197  
21    Forest Hill North & West, Forest Hill Road Park  43.696948 -79.411307  
22                      High Park, The Junction South  43.661608 -79.464763  
23                 North Toronto West,  Lawrence Park  43.715383 -79.405678  
24                The Annex, North Midtown, Yorkville  43.672710 -79.405678  
25                             Parkdale, Roncesvalles  43.648960 -79.456325  
26                                         Davisville  43.704324 -79.388790  
27                     University of Toronto, Harbord  43.662696 -79.400049  
28                                 Runnymede, Swansea  43.651571 -79.484450  
29                        Moore Park, Summerhill East  43.689574 -79.383160  
30          Kensington Market, Chinatown, Grange Park  43.653206 -79.400049  
31  Summerhill West, Rathnelly, South Hill, Forest...  43.686412 -

In [99]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [100]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [19]:
CLIENT_ID = 'MKKHBWPBK1ZI5ZVB3RC2AM2YEJEZX2JPR1XLG32WBJHCD4D1' # your Foursquare ID
CLIENT_SECRET = 'BR1IR1APQSF11WSGSRV13ZJQK0RWODXS1HTTVQVHH0YFGX5O' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MKKHBWPBK1ZI5ZVB3RC2AM2YEJEZX2JPR1XLG32WBJHCD4D1
CLIENT_SECRET:BR1IR1APQSF11WSGSRV13ZJQK0RWODXS1HTTVQVHH0YFGX5O


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [101]:
# type your answer here
print(toronto_data.columns)
toronto_venues = getNearbyVenues(toronto_data.Neighborhood, toronto_data.Latitude, toronto_data.Longitude)
toronto_venues.head()

Index(['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')
Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King a

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65426              -79.360636   
1  Regent Park, Harbourfront               43.65426              -79.360636   
2  Regent Park, Harbourfront               43.65426              -79.360636   
3  Regent Park, Harbourfront               43.65426              -79.360636   
4  Regent Park, Harbourfront               43.65426              -79.360636   

                    Venue  Venue Latitude  Venue Longitude  \
0        Roselle Desserts       43.653447       -79.362017   
1           Tandem Coffee       43.653559       -79.361809   
2  Cooper Koo Family YMCA       43.653249       -79.358008   
3     Body Blitz Spa East       43.654735       -79.359874   
4          Impact Kitchen       43.656369       -79.356980   

        Venue Category  
0               Bakery  
1          Coffee Shop  
2  Distribution Center  
3                  Spa  
4           Restaurant

In [102]:
toronto_venues[['Neighborhood', 'Venue Category']].groupby('Neighborhood').count()

Venue Category
Neighborhood                                                      
Berczy Park                                                     57
Brockton, Parkdale Village, Exhibition Place                    22
Business reply mail Processing Centre, South Ce...              14
CN Tower, King and Spadina, Railway Lands, Harb...              15
Central Bay Street                                              61
Christie                                                        16
Church and Wellesley                                            80
Commerce Court, Victoria Hotel                                 100
Davisville                                                      35
Davisville North                                                 8
Dufferin, Dovercourt Village                                    14
First Canadian Place, Underground city                         100
Forest Hill North & West, Forest Hill Road Park                  4
Garden District, Ryerson                                       100
Harbourfront East, Union Station, Toronto Islands              100
High Park, The Junction South                                   25
India Bazaar, The Beaches West                                  22
Kensington Market, Chinatown, Grange Park                       60
Lawrence Park                                                    4
Little Portugal, Trinity                                        43
Moore Park, Summerhill East                                      2
North Toronto West,  Lawrence Park                              19
Parkdale, Roncesvalles                                          14
Queen's Park, Ontario Provincial Government                     31
Regent Park, Harbourfront                                       46
Richmond, Adelaide, King                                        96
Rosedale                                                         4
Roselawn                                                         2
Runnymede, Swansea                                              36
St. James Town                                                  81
St. James Town, Cabbagetown                                     46
Stn A PO Boxes                                                  98
Studio District                                                 36
Summerhill West, Rathnelly, South Hill, Forest ...              14
The Annex, North Midtown, Yorkville                             20
The Beaches                                                      4
The Danforth West, Riverdale                                    43
Toronto Dominion Centre, Design Exchange                       100
University of Toronto, Harbord                                  33

In [103]:
print('There are %d of unique categories of venues' % len(toronto_venues['Venue Category'].unique()))

There are 233 of unique categories of venues


In [106]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
toronto_onehot.head()

Adult Boutique  Afghan Restaurant  Airport  Airport Food Court  \
0               0                  0        0                   0   
1               0                  0        0                   0   
2               0                  0        0                   0   
3               0                  0        0                   0   
4               0                  0        0                   0   

   Airport Gate  Airport Lounge  Airport Service  Airport Terminal  \
0             0               0                0                 0   
1             0               0                0                 0   
2             0               0                0                 0   
3             0               0                0                 0   
4             0               0                0                 0   

   American Restaurant  Antique Shop  ...  Tibetan Restaurant  \
0                    0             0  ...                   0   
1                    0             0  ...                   0   
2                    0             0  ...                   0   
3                    0             0  ...                   0   
4                    0             0  ...                   0   

   Toy / Game Store  Trail  Train Station  Vegetarian / Vegan Restaurant  \
0                 0      0              0                              0   
1                 0      0              0                              0   
2                 0      0              0                              0   
3                 0      0              0                              0   
4                 0      0              0                              0   

   Video Game Store  Vietnamese Restaurant  Wine Bar  Wine Shop  Yoga Studio  
0                 0                      0         0          0            0  
1                 0                      0         0          0            0  
2                 0                      0         0          0            0  
3                 0                      0         0          0            0  
4                 0                      0         0          0            0  

[5 rows x 233 columns]

In [107]:
toronto_onehot.shape

(1605, 233)

In [108]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Adult Boutique  \
0                                         Berczy Park          0.0000   
1        Brockton, Parkdale Village, Exhibition Place          0.0000   
2   Business reply mail Processing Centre, South C...          0.0000   
3   CN Tower, King and Spadina, Railway Lands, Har...          0.0000   
4                                  Central Bay Street          0.0000   
5                                            Christie          0.0000   
6                                Church and Wellesley          0.0125   
7                      Commerce Court, Victoria Hotel          0.0000   
8                                          Davisville          0.0000   
9                                    Davisville North          0.0000   
10                       Dufferin, Dovercourt Village          0.0000   
11             First Canadian Place, Underground city          0.0000   
12    Forest Hill North & West, Forest Hill Road Park          0.0000   
13                           Garden District, Ryerson          0.0000   
14  Harbourfront East, Union Station, Toronto Islands          0.0000   
15                      High Park, The Junction South          0.0000   
16                     India Bazaar, The Beaches West          0.0000   
17          Kensington Market, Chinatown, Grange Park          0.0000   
18                                      Lawrence Park          0.0000   
19                           Little Portugal, Trinity          0.0000   
20                        Moore Park, Summerhill East          0.0000   
21                 North Toronto West,  Lawrence Park          0.0000   
22                             Parkdale, Roncesvalles          0.0000   
23        Queen's Park, Ontario Provincial Government          0.0000   
24                          Regent Park, Harbourfront          0.0000   
25                           Richmond, Adelaide, King          0.0000   
26                                           Rosedale          0.0000   
27                                           Roselawn          0.0000   
28                                 Runnymede, Swansea          0.0000   
29                                     St. James Town          0.0000   
30                        St. James Town, Cabbagetown          0.0000   
31                                     Stn A PO Boxes          0.0000   
32                                    Studio District          0.0000   
33  Summerhill West, Rathnelly, South Hill, Forest...          0.0000   
34                The Annex, North Midtown, Yorkville          0.0000   
35                                        The Beaches          0.0000   
36                       The Danforth West, Riverdale          0.0000   
37           Toronto Dominion Centre, Design Exchange          0.0000   
38                     University of Toronto, Harbord          0.0000   

    Afghan Restaurant   Airport  Airport Food Court  Airport Gate  \
0              0.0000  0.000000            0.000000      0.000000   
1              0.0000  0.000000            0.000000      0.000000   
2              0.0000  0.000000            0.000000      0.000000   
3              0.0000  0.066667            0.066667      0.066667   
4              0.0000  0.000000            0.000000      0.000000   
5              0.0000  0.000000            0.000000      0.000000   
6              0.0125  0.000000            0.000000      0.000000   
7              0.0000  0.000000            0.000000      0.000000   
8              0.0000  0.000000            0.000000      0.000000   
9              0.0000  0.000000            0.000000      0.000000   
10             0.0000  0.000000            0.000000      0.000000   
11             0.0000  0.000000            0.000000      0.000000   
12             0.0000  0.000000            0.000000      0.000000   
13             0.0000  0.000000            0.000000      0.000000   
14             0.0000  0.000000            0.000000      0.000000   
15             0

In [109]:
toronto_grouped.shape

(39, 233)

In [110]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [139]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2  Business reply mail Processing Centre, South C...    Light Rail Station   
3  CN Tower, King and Spadina, Railway Lands, Har...      Airport Terminal   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Cocktail Bar              Beer Bar              Pharmacy   
1           Coffee Shop        Breakfast Spot                   Gym   
2                Garden           Pizza Place            Comic Shop   
3        Airport Lounge       Airport Service      Sculpture Garden   
4        Sandwich Place                  Café    Italian Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0            Restaurant        Farmers Market                Bakery   
1               Stadium         Burrito Place            Restaurant   
2            Restaurant         Burrito Place               Brewery   
3   Rental Car Location                 Plane       Harbor / Marina   
4           Salad Place       Bubble Tea Shop          Burger Joint   

  8th Most Common Venue      9th Most Common Venue      10th Most Common Venue  
0    Seafood Restaurant                Cheese Shop                       Hotel  
1          Climbing Gym      Performing Arts Venue                      Bakery  
2            Skate Park             Farmers Market        Fast Food Restaurant  
3         Boat or Ferry                        Bar                Airport Gate  
4       Thai Restaurant  Middle Eastern Restaurant  Modern European Restaurant

In [137]:
from sklearn.model_selection import GridSearchCV

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', axis=1)

grid = GridSearchCV(estimator=KMeans(n_init=100), param_grid={'n_clusters':[k for k in range(1,10)]})
grid.fit(toronto_grouped_clustering)
kmeans = grid.best_estimator_
n_clusters = len(kmeans.cluster_centers_)
labels = kmeans.labels_

print('Number of cluster centers: ', n_clusters)
print('labels: ', labels)

C:\Users\lawre\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Number of cluster centers:  4
labels:  [2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 0 2 1 2 2 2 2 2 0 3 2 2 2 2 2 2 2 1 2
 2 2]


In [140]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

Postal Code           Borough                                 Neighborhood  \
0         M5A  Downtown Toronto                    Regent Park, Harbourfront   
1         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2         M5B  Downtown Toronto                     Garden District, Ryerson   
3         M5C  Downtown Toronto                               St. James Town   
4         M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
0  43.654260 -79.360636               2           Coffee Shop   
1  43.662301 -79.389494               2           Coffee Shop   
2  43.657162 -79.378937               2           Coffee Shop   
3  43.651494 -79.375418               2           Coffee Shop   
4  43.676357 -79.293031               1     Health Food Store   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                Bakery                  Café                   Pub   
1      Sushi Restaurant           Yoga Studio   Fried Chicken Joint   
2        Clothing Store                  Café    Italian Restaurant   
3                  Café        Cosmetics Shop   American Restaurant   
4                 Trail                   Pub           Yoga Studio   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0                  Park        Breakfast Spot               Theater   
1            Restaurant                  Park             Nightclub   
2   Japanese Restaurant                 Hotel       Bubble Tea Shop   
3          Cocktail Bar           Cheese Shop            Restaurant   
4         Deli / Bodega           Escape Room     Electronics Store   

         8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0                  Yoga Studio        Cosmetics Shop             Shoe Store  
1                  Music Venue    Mexican Restaurant    Japanese Restaurant  
2    Middle Eastern Restaurant        Cosmetics Shop       Ramen Restaurant  
3           Italian Restaurant   Japanese Restaurant       Department Store  
4  Eastern European Restaurant            Donut Shop       Doner Restaurant

In [141]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(n_clusters)
ys = [i + x + (i*x)**2 for i in range(n_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [142]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
18  Lawrence Park      Business Service                  Park   
33       Rosedale                  Park            Playground   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
18           Swim School              Bus Line           Yoga Studio   
33                 Trail           Yoga Studio         Deli / Bodega   

   6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
18           Escape Room     Electronics Store  Eastern European Restaurant   
33           Escape Room     Electronics Store  Eastern European Restaurant   

   9th Most Common Venue 10th Most Common Venue  
18            Donut Shop       Doner Restaurant  
33            Donut Shop       Doner Restaurant

In [143]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

Neighborhood 1st Most Common Venue  \
4                                       The Beaches     Health Food Store   
21  Forest Hill North & West, Forest Hill Road Park         Jewelry Store   
29                      Moore Park, Summerhill East                 Trail   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
4                  Trail                   Pub           Yoga Studio   
21                 Trail    Mexican Restaurant      Sushi Restaurant   
29          Tennis Court           Yoga Studio         Deli / Bodega   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
4          Deli / Bodega           Escape Room     Electronics Store   
21           Yoga Studio      Department Store           Escape Room   
29  Ethiopian Restaurant           Escape Room     Electronics Store   

          8th Most Common Venue        9th Most Common Venue  \
4   Eastern European Restaurant                   Donut Shop   
21            Electronics Store  Eastern European Restaurant   
29  Eastern European Restaurant                   Donut Shop   

   10th Most Common Venue  
4        Doner Restaurant  
21             Donut Shop  
29       Doner Restaurant

In [145]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

Neighborhood 1st Most Common Venue  \
0                           Regent Park, Harbourfront           Coffee Shop   
1         Queen's Park, Ontario Provincial Government           Coffee Shop   
2                            Garden District, Ryerson           Coffee Shop   
3                                      St. James Town           Coffee Shop   
5                                         Berczy Park           Coffee Shop   
6                                  Central Bay Street           Coffee Shop   
7                                            Christie         Grocery Store   
8                            Richmond, Adelaide, King           Coffee Shop   
9                        Dufferin, Dovercourt Village              Pharmacy   
10  Harbourfront East, Union Station, Toronto Islands           Coffee Shop   
11                           Little Portugal, Trinity                   Bar   
12                       The Danforth West, Riverdale      Greek Restaurant   
13           Toronto Dominion Centre, Design Exchange           Coffee Shop   
14       Brockton, Parkdale Village, Exhibition Place                  Café   
15                     India Bazaar, The Beaches West                  Park   
16                     Commerce Court, Victoria Hotel           Coffee Shop   
17                                    Studio District           Coffee Shop   
20                                   Davisville North          Dance Studio   
22                      High Park, The Junction South       Thai Restaurant   
23                 North Toronto West,  Lawrence Park        Clothing Store   
24                The Annex, North Midtown, Yorkville        Sandwich Place   
25                             Parkdale, Roncesvalles             Gift Shop   
26                                         Davisville           Pizza Place   
27                     University of Toronto, Harbord                  Café   
28                                 Runnymede, Swansea                  Café   
30          Kensington Market, Chinatown, Grange Park                  Café   
31  Summerhill West, Rathnelly, South Hill, Forest...           Coffee Shop   
32  CN Tower, King and Spadina, Railway Lands, Har...      Airport Terminal   
34                                     Stn A PO Boxes           Coffee Shop   
35                        St. James Town, Cabbagetown           Pizza Place   
36             First Canadian Place, Underground city           Coffee Shop   
37                               Church and Wellesley   Japanese Restaurant   
38  Business reply mail Processing Centre, South C...    Light Rail Station   

            2nd Most Common Venue   3rd Most Common Venue  \
0                          Bakery                    Café   
1                Sushi Restaurant             Yoga Studio   
2                  Clothing Store                    Café   
3                            Café          Cosmetics Shop   
5                    Cocktail Bar                Beer Bar   
6                  Sandwich Place                    Café   
7                            Café                    Park   
8                            Café              Restaurant   
9                          Bakery  Furniture / Home Store   
10                       Aquarium                   Hotel   
11               Asian Restaurant              Restaurant   
12             Italian Restaurant             Coffee Shop   
13                          Hotel                    Café   
14                    Coffee Shop          Breakfast Spot   
15           Fast Food Restaurant                     Gym   
16                     Restaurant                    Café   
17                         Bakery               Gastropub   
20                           Park    Gym / Fitness Center   
22                           Café      Mexican Restaurant   
23                    Coffee Shop             Yoga Studio   
24                           Café             Coffee Shop   
25          

In [144]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
19     Roselawn           Music Venue                Garden   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
19          Dance Studio           Escape Room     Electronics Store   

          6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
19  Eastern European Restaurant            Donut Shop      Doner Restaurant   

   9th Most Common Venue 10th Most Common Venue  
19               Dog Run    Distribution Center

There are <b>four</b> clusters:
<ol>
    <li>characterized by open space like <b>parks</b> and <b>trail</b></li>
    <li>characterized by fitness facility like <b>trail</b> and <b>Tennis court</b>
    <li>characterized by dining places like <b>coffee shop</b> and <b>Café</b> and <b>restaurants</b></li>
    <li>characterized by leisure places like <b>music venue</b>, <b>Garden</b> and <b>Dance studio</b></li>
</ol 